<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Collagen-Pradeep/Bioco_Collagen_Klassiske_Superveiledet_Regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Importering, lasting og formattering

Lasting av nødvendige bibliotek og pakker

In [2]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 4.8 MB/s eta 0:00:00


Importering av nødvendig bibliotek og pakker

In [3]:
import pandas as pd
import numpy as np
import copy
import math
from statistics import mean, stdev

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline

In [4]:
# Modeller
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

In [5]:
# Hyperoptimalisering
import optuna
from optuna.visualization import plot_optimization_history

# Utelukker utskriving av logg for hyperoptimalisering
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Egendefinerte moduler

In [6]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_trening_validering_testing_evaluering.py /content/moduler_trening_validering_testing_evaluering.py

/content


In [7]:
from moduler_trening_validering_testing_evaluering import trening_testsett_oppdeling_enzym
from moduler_trening_validering_testing_evaluering import trening_testsett_oppdeling_dag
from moduler_trening_validering_testing_evaluering import trening_validering_oppdeling_dag

from moduler_trening_validering_testing_evaluering import enzymfordeling
from moduler_trening_validering_testing_evaluering import n_splitt_enzym
from moduler_trening_validering_testing_evaluering import Resultat_evaluering
from moduler_trening_validering_testing_evaluering import res_eval_opt
from moduler_trening_validering_testing_evaluering import res_eval
from moduler_trening_validering_testing_evaluering import Resultat_dicts

In [8]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_optimalisering_evaluering_veiledet.py /content/moduler_optimalisering_evaluering_veiledet.py

/content


In [9]:
from moduler_optimalisering_evaluering_veiledet import trening_rfr_enzym
from moduler_optimalisering_evaluering_veiledet import trening_knr_enzym
from moduler_optimalisering_evaluering_veiledet import trening_svr_enzym

from moduler_optimalisering_evaluering_veiledet import trening_rfr_dag
from moduler_optimalisering_evaluering_veiledet import trening_knr_dag
from moduler_optimalisering_evaluering_veiledet import trening_svr_dag

from moduler_optimalisering_evaluering_veiledet import veiledet_modell_evaluering_1
from moduler_optimalisering_evaluering_veiledet import veiledet_modell_evaluering_2

# Importering av relevant data

In [10]:
# Velger første kolonne med dato og tid som index
data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data.csv',
                            header=0,
                            sep=',',
                            index_col=0)

# Design produksjon
data_design = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data_design.csv',
                                   header=0,
                                   sep=',',
                                   index_col=0)

# Importerer rådata
rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
data.index = pd.to_datetime(data.index,
                            format='%Y-%m-%d %H:%M:%S')

data_design.index = pd.to_datetime(data_design.index,
                                   format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

# Lagring av resultater

In [11]:
# Oppretter instans for lagring av resultater
res = Resultat_dicts()

# Parametere til hyperoptimalisering

In [12]:
# Antall runder til optimalisering
n_runder = 100

# Parametervalg for kryssvalidering
n_gjentagelser = 5

Tilfeldighetsfrø

In [13]:
#Tilfeldighetsfrø
random_seed = 2024

# NIR målinger inkludert

## Hele datasett

## Alternativ 1: 25% Testsett og Hele resterende treningssett

In [14]:
# Setter fast testandel og treningsandel
test_andel = 0.25
trenings_andel = 1.0


# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _, _, _, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data,
                                   test_andel=test_andel)

In [ ]:
# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(dict_res_opt=res.dict_res_1_hel_opt,
                             dict_res=res.dict_res_1_hel,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_gjentagelser=n_gjentagelser,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

Lagring av resultater

In [ ]:
res_1_hel = pd.DataFrame(res.dict_res_1_hel)
res_1_hel_opt = pd.DataFrame(res.dict_res_1_hel_opt)

res_1_hel.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_resultater_1_hel.csv')
res_1_hel_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_resultater_1_hel_opt.csv')

Velger beste modell

## Alternativ 2: 25% Testsett og resteremde hele treningssett

In [14]:
# Definerer trenings- og testandel
test_andel = 0.25
trenings_andel = 1.0

# Deler datasettet i trening og testsett
trening_markert, testsett, _, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

In [ ]:
# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(dict_res_opt=res.dict_res_2_hel_opt,
                             dict_res=res.dict_res_2_hel,
                             rå_data=rå_data,
                             treningssett_markert=trening_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

Lagring av resultater

In [ ]:
res_2_hel = pd.DataFrame(res.dict_res_2_hel)
res_2_hel_opt = pd.DataFrame(res.dict_res_2_hel_opt)

res_2_hel.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_resultater_2_hel.csv')
res_2_hel_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_resultater_2_hel_opt.csv')

Velger beste modell

## Designproduksjon

## Alternativ 1: 25% Testsett og Hele resterende treningssett

In [ ]:
# Setter fast testandel og treningsandel
test_andel = 0.25
trenings_andel = 1.0

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _, _, _, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design,
                                   test_andel=test_andel)

In [ ]:
# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(dict_res_opt=res.dict_res_1_design_opt,
                             dict_res=res.dict_res_1_design,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_gjentagelser=n_gjentagelser,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

Lagring av resultater

In [ ]:
res_1_design = pd.DataFrame(res.dict_res_1_design)
res_1_design_opt = pd.DataFrame(res.dict_res_1_design_opt)

res_1_design.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_resultater_1_design.csv')
res_1_design_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_resultater_1_design_opt.csv')

Velger beste modell

## Alternativ 2: 25% Testsett og resteremde hele treningssett

In [ ]:
# Definerer trenings- og testandel
test_andel = 0.25
trenings_andel = 1.0


# Deler datasettet i trening og testsett
trening_markert, testsett, _, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Enzymtyper Fordeling: Test
            Frekvens  Andel
EnzymeCode                 
A2                25  27.78
B                 16  17.78
C                 15  16.67
E                 15  16.67
D                 13  14.44
A1                 6   6.67


In [ ]:
# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(dict_res_opt=res.dict_res_2_design_opt,
                             dict_res=res.dict_res_2_design,
                             rå_data=rå_data,
                             treningssett_markert=trening_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Lagring av resultater

In [ ]:
res_2_design = pd.DataFrame(res.dict_res_2_design)
res_2_design_opt = pd.DataFrame(res.dict_res_2_design_opt)

res_2_design.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_resultater_2_design.csv')
res_2_design_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_resultater_2_design_opt.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Velger beste modell

# NIR målinger ekskludert

In [ ]:
# Ekskluderer NIR målinger fra datasettene
NIR_kolonner = ['NIRfat', 'NIRwater', 'NIRash']
data_uten_NIR = data.drop(columns=NIR_kolonner)
data_design_uten_NIR = data_design.drop(columns=NIR_kolonner)

In [ ]:
# Setter fast testandel og treningsandel
test_andel = 0.25
trenings_andel = 1.0

## Hele datasett

## Alternativ 1: 25% Testsett og Hele resterende treningssett

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _, _, _, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_uten_NIR,
                                   test_andel=test_andel)

# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(dict_res_opt=res.dict_res_1_hel_opt_NIR,
                             dict_res=res.dict_res_1_hel_NIR,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_gjentagelser=n_gjentagelser,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)



Lagring av resultater

In [ ]:
res_1_hel_NIR = pd.DataFrame(res.dict_res_1_hel_NIR)
res_1_hel_opt_NIR = pd.DataFrame(res.dict_res_1_hel_opt_NIR)

res_1_hel_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_resultater_1_hel_NIR.csv')
res_1_hel_opt_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_resultater_1_hel_opt_NIR.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Velger beste modell

## Alternativ 2: 25% Testsett og resteremde hele treningssett

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, testsett, _, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_uten_NIR,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(dict_res_opt=res.dict_res_2_hel_opt_NIR,
                             dict_res=res.dict_res_2_hel_NIR,
                             rå_data=rå_data,
                             treningssett_markert=trening_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Enzymtyper Fordeling: Test
            Frekvens  Andel
EnzymeCode                 
A2                25  27.78
B                 16  17.78
C                 15  16.67
E                 15  16.67
D                 13  14.44
A1                 6   6.67


Lagring av resultater

In [ ]:
res_2_hel_NIR = pd.DataFrame(res.dict_res_2_hel_NIR)
res_2_hel_opt_NIR = pd.DataFrame(res.dict_res_2_hel_opt_NIR)

res_2_hel_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_resultater_2_hel_NIR.csv')
res_2_hel_opt_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_resultater_2_hel_opt_NIR.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Velger beste modell

## Designsett

## Alternativ 1: 25% Testsett og Hele resterende treningssett

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _, _, _, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design_uten_NIR,
                                   test_andel=test_andel)
# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(dict_res_opt=res.dict_res_1_design_opt_NIR,
                             dict_res=res.dict_res_1_design_NIR,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_gjentagelser=n_gjentagelser,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

Lagring av resultater

In [ ]:
res_1_design_NIR = pd.DataFrame(res.dict_res_1_design_NIR)
res_1_design_opt_NIR = pd.DataFrame(res.dict_res_1_design_opt_NIR)

res_1_design_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_resultater_1_design_NIR.csv')
res_1_design_opt_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_resultater_1_design_opt_NIR.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Velger beste modell

## Alternativ 2: 25% Testsett og resteremde hele treningssett

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, testsett, _, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(dict_res_opt=res.dict_res_2_design_opt_NIR,
                             dict_res=res.dict_res_2_design_NIR,
                             rå_data=rå_data,
                             treningssett_markert=trening_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Enzymtyper Fordeling: Test
            Frekvens  Andel
EnzymeCode                 
A2                25  27.78
B                 16  17.78
C                 15  16.67
E                 15  16.67
D                 13  14.44
A1                 6   6.67


Lagring av resultater

In [ ]:
res_2_design_NIR = pd.DataFrame(res.dict_res_2_design_NIR)
res_2_design_opt_NIR = pd.DataFrame(res.dict_res_2_design_opt_NIR)

res_2_design_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_resultater_2_design_NIR.csv')
res_2_design_opt_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/collagen/collagen_resultater_2_design_opt_NIR.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Velger beste modell